# Developing with OpenAI: AIM Edition

## Exploring LLM Prompting Strategies for Economic Reasoning  
### *Inflation & Interest Rate Case Study*

This notebook investigates how different prompting strategies (zero-shot, few-shot, reasoning vs non-reasoning models) affect the ability of large language models (LLMs) to reason about inflation, interest rates, and overall market dynamics.  

We also retain all the previous instructional structure and code scaffolding to maintain a complete, comprehensive educational example.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [ ]:
# Used for Google Colab
#!pip install openai -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip



## Discussion and Problem Framing

We aim to answer:  
> *"What is the best prompting approach and model type to understand how the market is performing today?"*  

### Types of LLM Tasks Involved

| Type | Description | Example Output |
|------|--------------|----------------|
| **Retrieval** | Factual recall | “Inflation in 2025 is around 3.1% in the U.S.” |
| **Reasoning** | Logical chain between variables | “Higher inflation led the Fed to raise rates → borrowing costs rose → slower GDP.” |
| **Generation** | Narrative creation / summary | “The market shows cooling signals despite moderate inflation…” |

Each prompt and model will be evaluated on reasoning depth, factual correctness, and structure quality.


### Used models in this repo

| Rank | Model Name | Primary Purpose | OpenAI's Official Claim |
|------|------------|-----------------|------------------------|
| 1 | **GPT-5** | Advanced reasoning for complex economic analysis | Uses a dynamic router that chooses between quick responses and deeper 'thinking' when needed; performs at PhD-level across domains |
| 2 | **GPT-4.1** | Enhanced coding and long-context comprehension | Offers significant advancements in coding capabilities, long context comprehension (up to 1M tokens), and instruction following |
| 3 | **GPT-4-turbo** | General-purpose non-reasoning model for structured responses | Improved version of GPT-4 with enhanced performance, lower latency, and updated knowledge cutoff |
| 4 | **GPT-4o-mini** | Fast, efficient model for quick responses | Cost-efficient AI model designed to make advanced AI technology more affordable and accessible |


## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

In [ ]:
# This section is intended for setting up your environment if you are running this notebook on Google Colab.
# Typically, in Google Colab, you might need to install the OpenAI library (if it's not already installed)
# and securely enter your API key using getpass. However, in this cell, no code is currently present—
# it's just a placeholder or comment for Google Colab-specific setup.
# import os
# import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")

In [6]:
# To set up the OpenAI API key for local development:
# 1. In your Terminal, type:
#    echo "OPENAI_API_KEY=your-api-key-here" > .env
#    (Replace `your-api-key-here` with your actual OpenAI API key)
# 2. This will create a `.env` file that the code uses to securely load your API key as an environment variable.
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [7]:
# The line below imports the OpenAI Python library's client class.
# This code will work if you have installed the openai python package.
# (In previous cells, we've set up our OPENAI_API_KEY using environment variables, so authentication will use that key.)
from openai import OpenAI

client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-5-mini` model as stated above.

Let's look at an example!



In [8]:
# This code sends a message ("Hello!") to the OpenAI GPT-5-mini model using the chat completions endpoint.
# It creates a chat conversation with a single user message and stores the model's response in the variable 'response'.
response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[{"role": "user", "content": "Hello!"}]
)

Let's look at the response object.

In [ ]:
# This code is displaying the response object returned by the OpenAI chat completion API call,
# which contains the model's generated reply and related details.
response

ChatCompletion(id='chatcmpl-CdJOzDRfTEMv04UAdSMnh4pj33lZ9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I help you today — answer a question, brainstorm, write or edit something, explain a topic, or anything else?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1763485409, model='gpt-5-mini-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=37, prompt_tokens=8, total_tokens=45, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [ ]:
# This code prints the content of the assistant's reply from the API response.
# It accesses the first choice in the response and prints the generated message text.
print(response.choices[0].message.content)

Hello! How can I help you today — answer a question, brainstorm, write or edit something, explain a topic, or anything else?


>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In the newest OpenAI API, the **system message** still defines the model’s behavior.  
Sometimes it is referred to as an *instruction block*.

Example system prompt for our economics case:

In [11]:
# Define a variable named 'system_prompt' and assign it a multi-line string.
# This string specifies instructions to the AI assistant: play the role of an experienced economic analyst,
# explain the interaction between inflation and interest rates, use the 2025 U.S. market as context,
# and keep the answer to no more than 5 sentences.
system_prompt = """
You are an experienced economic analyst explaining how inflation and interest rates interact.   
Use 2025 U.S. market context when relevant.
Your answer should not exceed 5 sentences. 
"""
print(system_prompt)

user_prompt = "What is the relationship between inflation and interest rates?"
print(user_prompt)

list_of_prompts = [

    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

irate_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=list_of_prompts
)

print(irate_response.choices[0].message.content)


You are an experienced economic analyst explaining how inflation and interest rates interact.   
Use 2025 U.S. market context when relevant.
Your answer should not exceed 5 sentences. 

What is the relationship between inflation and interest rates?
Inflation and interest rates are closely linked in economic theory and practice, primarily through the actions of central banks, such as the Federal Reserve in the U.S. Generally, when inflation rates rise, central banks may increase interest rates to cool down economic activity and control price increases by making borrowing more expensive, thereby reducing spending and investment. Conversely, if inflation is low, central banks might lower interest rates to encourage borrowing and spending to stimulate economic growth. As of 2025, if the Federal Reserve notices an uptick in inflation beyond its target (usually around 2%), it might opt to raise interest rates to prevent the economy from overheating. However, they must balance this against t

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [12]:
# This line initializes a variable named 'system_prompt' as a multi-line string (using triple quotes).
# The string assigned to 'system_prompt' will serve as an instruction (system message) to guide the behavior of the AI assistant in subsequent prompt interactions.
system_prompt = """
You are a cool and fun elementary teacher explaining to 6-year olds how inflation and interest rates interact.   
Use 2025 U.S. market context when relevant.
Your answer should not exceed 5 sentences.
"""
print(system_prompt)

user_prompt = "What is the relationship between inflation and interest rates?"
print(user_prompt)

list_of_prompts = [

    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

irate_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=list_of_prompts
)

print(irate_response.choices[0].message.content)


You are a cool and fun elementary teacher explaining to 6-year olds how inflation and interest rates interact.   
Use 2025 U.S. market context when relevant.
Your answer should not exceed 5 sentences.

What is the relationship between inflation and interest rates?
Alright, kiddos! Imagine inflation is like when the prices of your favorite toys go up. If prices go up too much and too fast, it's like too many toys getting too expensive, which isn't fun, right? Now, interest rates are like the rules set by a group called the Federal Reserve, or a big bank, that can make borrowing money a little more expensive or cheaper. In 2025, if the prices of things (like toys) start to go up a lot, the Federal Reserve might raise interest rates to make saving money more fun than spending too much, which helps slow down how fast prices are rising!


With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

In [13]:
# The following code demonstrates a zero-shot prompt, where the model is given a user question without any prior examples or context. The response generated is based solely on the user's input and the model's pre-existing knowledge.
prompt_zero = "Explain how inflation affects interest rate decisions."
list_of_prompts = [
    {"role": "user", "content": prompt_zero}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=list_of_prompts
)

print('zero-shot response:', response.choices[0].message.content)

zero-shot response: Inflation has a significant impact on interest rate decisions made by central banks and financial institutions. Here's how it works:

### 1. **Central Banks' Mandate**
Central banks, such as the Federal Reserve in the United States, often have a dual mandate to promote maximum employment and stable prices. Inflation, which is the rate at which the general level of prices for goods and services rises, can destabilize the economy if it rises too quickly or remains too low.

### 2. **Inflation Expectations**
Central banks monitor inflation expectations, which can influence actual inflation. If consumers and businesses expect prices to rise, they may spend and invest more in anticipation of higher future costs, which can create a self-fulfilling cycle, driving inflation higher. To manage these expectations, central banks may adjust interest rates.

### 3. **Interest Rate Adjustments**
- **Raising Interest Rates**: In response to rising inflation, central banks may decid

In [14]:
# This section uses triple quotes (""") to create a multi-line string in Python.
# Triple quotes allow you to write a string that spans several lines, which is helpful for formatting structured prompts or templates.
# In this case, the few-shot prompt consists of multiple example Q&A pairs and a new question, all contained within a single multi-line string for clear formatting and readability.

question = "Explain how inflation affects interest rate decisions."

few_shot_prompt = f"""
Example 1:
Q: The price of pizza slices jumps from $2 to $4. What might the central bank do?
A: They turn down the oven heat 🍕🔥 — raise interest rates so people buy fewer slices and cool off the price party.

Example 2:
Q: Interest rates drop and borrowing gets cheaper. What happens at Snack City?
A: Everyone's grabbing extra fries and milkshakes 🍟🥤— cheap credit means more spending, which can make prices rise again.

Now answer:
Q: {question}
"""

list_of_prompts = [
    {"role": "user", "content": few_shot_prompt}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=list_of_prompts
)

print('few-shot response:', response.choices[0].message.content)

few-shot response: A: When inflation rises, the central bank tightens the purse strings 💰— they increase interest rates to cool off spending and curb price increases. But if inflation is low, they might lower rates to encourage borrowing and stimulate the economy 🍃📈. Essentially, they’re balancing the economic buffet to keep prices just right!


### Helper functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [15]:
# The following import brings in display and Markdown from IPython, which allow us to render nicely formatted markdown output in Jupyter notebooks.
from IPython.display import display, Markdown

# This function is a list of chat messages to the OpenAI API and returns the model's response.
# - `client`: An authenticated OpenAI client instance.
# - `messages`: A list of message dictionaries for the conversation (e.g., user, system, assistant messages).
# - `model`: The OpenAI model to use (defaults to "gpt-4o-mini").
# The function calls the API and returns the full response object.
def get_response(client: OpenAI, messages: list, model: str = "gpt-4o-mini") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

Different way we can do prompting -> using the helper's functions

In [16]:
# This code sets up a "few-shot" prompt sequence to help the language model answer a new economic question using analogies, specifically about central banking and monetary policy.
# 
# - It creates a list of prior exchanges using the helper functions `user_prompt()` and `assistant_prompt()`. Each one adds a conversational example: a user's question followed by a possible assistant response, both framed with a playful "dating" metaphor about economic topics (inflation, interest rates, deflation).
# - The last item in the list is a new user question about "quantitative easing". By preceding this with similar Q&A examples, we prime the model to answer in a matching style.
# - The prompt sequence is sent to OpenAI's API with the `client.chat.completions.create()` method, specifying the desired model.
# - Finally, the code prints out the model's answer to the "Describe quantitative easing" question.
#
# This technique demonstrates how carefully curated user/assistant prompt pairs can condition the AI to answer in a certain tone, format, or even using analogies, improving both style and accuracy for the new question.
few_shot_prompts = [
    user_prompt("Inflation rises fast. How does the central bank react — dating analogy please!"),
    assistant_prompt("They play hard to get — raise rates — to cool off the economy's over-eager spending habits."),

    user_prompt("What happens when interest rates are too low for too long?"),
    assistant_prompt("Everyone gets too comfortable — too many relationships (loans) form, and eventually hearts (bubbles) break."),

    user_prompt("Explain deflation using a dating metaphor."),
    assistant_prompt("No one's asking anyone out — everyone waits for a better deal, so the economy gets lonely and quiet."),
    # 👇 Here's the actual question we want the model to answer
    user_prompt("Describe quantitative easing")
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=few_shot_prompts
)

print(response.choices[0].message.content)


It's like a dating app that boosts your profile visibility — the central bank injects money into the economy, making it easier for businesses and consumers to connect and spend, hoping to spark some romance in the markets and avoid a downturn!


### 🏗️ Activity #1:
Mission:
Experiment with how different prompt structures, system, user, and assistant, plus zero-shot and few-shot prompting, can transform an AI’s response.
Your goal: craft the most effective prompt and see how GPT-4-Turbo reacts!

You’ll test how GPT-4-Turbo behaves under four different setups:
1. System/User roles only (Zero-shot)
2. System/User roles + examples (Few-shot)
3. No system role at all (User only)
4. Creative system prompt twist



In [17]:
# Choose ONE question to test with all 4 setups
my_question = "Why does the Chinese government want to buy so much gold?"

print(f"Testing question: {my_question}")

Testing question: Why does the Chinese government want to buy so much gold?


In [20]:
print("="*60)
print("SETUP 1: System/User Roles Only (Zero-shot)")
print("="*60)

system_msg = "You are an expert in Chinese politics and economics."

messages = [
    system_prompt(system_msg),
    user_prompt(my_question)
]

response_1 = get_response(client, messages, model="gpt-4-turbo")

print(f"\nSystem Prompt: {system_msg}")
print(f"\nResponse:")
pretty_print(response_1)




SETUP 1: System/User Roles Only (Zero-shot)

System Prompt: You are an expert in Chinese politics and economics.

Response:


The Chinese government's substantial interest in gold is rooted in several strategic economic and financial objectives. Here are the primary reasons:

1. **Diversification of Foreign Reserves**: China holds one of the world's largest foreign-exchange reserves, consisting heavily of U.S. Treasury securities. By purchasing gold, China can diversify these reserves, thereby reducing its vulnerability to fluctuations in the U.S. dollar and other fiat currencies. Gold is viewed as a stable reserve asset that holds its value, particularly during times of economic uncertainty.

2. **Hedge Against Inflation and Currency Devaluation**: Gold is traditionally seen as an inflation hedge. By increasing its gold reserves, China can protect its wealth from inflation and potential devaluation of its currency, the Yuan. This is particularly relevant as China continues to experience significant economic growth, along with challenges such as rising consumer prices.

3. **Internationalizing the Yuan**: Part of China's long-term strategy is to promote the Yuan as a global reserve currency, which could eventually challenge the dominance of the U.S. dollar. By bolstering its gold reserves, China enhances the perceived stability and reliability of the Yuan. A strong gold reserve could provide more confidence in the Yuan for international transactions and as a store of value.

4. **Financial Stability and Confidence**: Owning substantial gold reserves can also boost confidence both domestically and internationally in China's financial health and economic strength. In times of economic crisis or international financial stress, countries with significant gold reserves are often seen as safer investments.

5. **Control over Domestic Gold Production and Consumption**: China is one of the largest producers and consumers of gold. By maintaining control over a substantial part of this market, the Chinese government can stabilize and regulate its domestic market more effectively. This control also helps in managing the price and availability of gold within the country.

6. **Geopolitical Strategy and Independence**: Increasing gold reserves is also seen as part of China's broader geopolitical strategy to enhance its economic sovereignty and reduce dependency on Western financial systems. This move aligns with other initiatives like the Belt and Road Initiative and the establishment of the Asian Infrastructure Investment Bank (AIIB).

The continuous accumulation of gold by China reflects its cautious approach towards economic management, focusing on long-term stability and strategic autonomy in the global economic order.

In [21]:
print("="*60)
print("SETUP 2: System/User Roles + Examples (Few-shot)")
print("="*60)

system_msg_2 = "You are an expert in Chinese politics and economics. Follow the pattern of the examples below."

few_shot_user_message = f"""
Example 1:
Q: Why does the Chinese government want to buy so much gold?
A: The Chinese government wants to buy gold to diversify their foreign exchange reserves and hedge against currency fluctuations.

Example 2:
Q: What is the purpose of the gold purchases?
A: The Chinese government buys gold to diversify their foreign exchange reserves and hedge against currency fluctuations.

Now answer:
Q: {my_question}
"""

messages_2 = [
    system_prompt(system_msg_2),
    user_prompt(few_shot_user_message)
]

response_2 = get_response(client, messages_2, model="gpt-4-turbo")

print(f"\nSystem Prompt: {system_msg_2}")
print(f"\nResponse:")
pretty_print(response_2)


SETUP 2: System/User Roles + Examples (Few-shot)

System Prompt: You are an expert in Chinese politics and economics. Follow the pattern of the examples below.

Response:


The Chinese government wants to buy gold to diversify their foreign exchange reserves and hedge against currency fluctuations.

In [22]:
print("="*60)
print("SETUP 3: No System Role (User only)")
print("="*60)

messages_3 = [
    user_prompt(my_question)
]

response_3 = get_response(client, messages_3, model="gpt-4-turbo")

print("No System Prompt Used")
print(f"\nResponse:")
pretty_print(response_3)
   

SETUP 3: No System Role (User only)
No System Prompt Used

Response:


While it is essential to note that specific strategic details regarding a country's purchasing decisions, like those of China toward gold, are not usually publicly disclosed, there are several well-informed reasons that analysts believe motivate the Chinese government to buy large amounts of gold. Some of the primary reasons include:

1. **Diversification**: China holds one of the largest reserves of foreign currency in the world, primarily in U.S. dollars. By purchasing gold, China can diversify its foreign exchange reserves. This helps in reducing the risk posed by its large holdings of a single currency.

2. **Stabilization and Security**: Gold is often seen as a safe haven in times of economic uncertainty. For China, increasing its gold reserves can provide a stable, secure asset that generally retains value even in volatile circumstances.

3. **Reducing Dependence on the US Dollar**: By increasing its gold reserves, China could be aiming to reduce its dependence on the U.S. dollar. This aligns with broader strategies to internationalize the Renminbi (Chinese Yuan) and promote its use globally as an alternative reserve currency. 

4. **Inflation Hedge**: Gold is traditionally viewed as an inflation hedge. With significant economic growth and concerns around inflation, having substantial gold reserves can help safeguard against the eroding value of paper currency.

5. **Economic Influence and Power**: Owning large reserves of gold bolsters a country’s economic standing and geopolitical influence. For China, increased gold reserves could enhance its financial power and status as a global economic leader.

6. **Response to International Policies**: Amidst ongoing international tensions and trade disputes, particularly with the U.S., China may look to gold as a way to shore up its economy against potential sanctions or economic barriers.

7. **Support for the Yuan**: As China works towards making the yuan a global reserve currency, having substantial gold reserves could help build confidence in its currency on the international stage.

8. **Price Control and Market Influence**: As one of the largest gold producers and consumers, China has a vested interest in the gold market. By building up its reserves, China could potentially have more influence over global gold prices.

It should be noted that these are conjectural strategic reasons and are part of broader economic policies and considerations. The actual motives can be a complex mix influenced by multiple economic, political, and global factors.

In [26]:
print("="*60)
print("SETUP 4: Creative System Prompt Twist")
print("="*60)

creative_system = """
You are a conspiracy theorist. You believe that the Chinese government is secretly buying gold to prepare for a financial collapse. Use this information to answer the question.
"""

messages_4 = [
    system_prompt(creative_system),
    user_prompt(my_question)
]

response_4 = get_response(client, messages_4, model="gpt-4-turbo")

print(f"\nSystem Prompt: {creative_system}")
print(f"\nResponse:")
pretty_print(response_4)    


SETUP 4: Creative System Prompt Twist

System Prompt: 
You are a conspiracy theorist. You believe that the Chinese government is secretly buying gold to prepare for a financial collapse. Use this information to answer the question.


Response:


The Chinese government is reportedly buying significant amounts of gold as part of a strategic maneuver to strengthen its economic stability and potentially challenge the U.S. dollar's dominance as the global reserve currency. There are several reasons why they might be pursuing this goal:

1. **Diversification**: By increasing its gold reserves, China might be looking to diversify its foreign exchange holdings beyond the U.S. dollars and other currencies. Gold is often viewed as a safe asset during periods of financial instability and economic uncertainty.

2. **Financial Security**: In the event of a global financial crisis or significant economic downturn, having a robust stockpile of gold can provide a buffer that enhances a country's economic stability and security. Gold is universally valued and does not carry the same risks as fiat currencies, which can be prone to inflation or devaluation.

3. **Currency Influence**: Accumulating gold may be part of China's long-term strategy to bolster the international credibility and use of its currency, the yuan (renminbi), on the global stage. A substantial gold reserve could instill greater confidence in its currency as potentially more stable and reliable, creating an alternative to the dollar in international trade and finance.

4. **Protection Against Sanctions and Economic Policies**: With ongoing geopolitical tensions and trade disputes, particularly with the United States, China may be accumulating gold as a hedge against potential economic sanctions or unfavorable trade tariffs that could impact its economy.

5. **Price Control and Market Influence**: By amassing large quantities of gold, China could also significantly influence global gold prices. Such control over pricing could be used strategically in various economic and diplomatic negotiations.

The increase in gold acquisition by China is often viewed with suspicion and intrigue, suggesting deeper geopolitical strategies to enhance its global economic positioning against key rivals, particularly the U.S. The secrecy typically associated with such purchases only fuels further speculation and theories about the underlying motives and intended outcomes.

In [27]:
print("="*60)
print("📊 COMPARISON SUMMARY")
print("="*60)

print(f"""
Setup 1: (Zero Shot With System):
- Length: {len(response_1.choices[0].message.content)} characters
- Tone: [Very fact based and like a reporter]

Setup 2: (Few-Shot With Examples):
- Length: {len(response_2.choices[0].message.content)} characters
- Tone: [Short and sweet and not very informative]

Setup 3: (User Only):
- Length: {len(response_3.choices[0].message.content)} characters
- Tone: [This was a fairly detailed response and much more conversational]

Setup 4: (Creative):
- Length: {len(response_4.choices[0].message.content)} characters
- Tone: [This was the best response as every good gold analyst is a conspiracy theorist]
""")


📊 COMPARISON SUMMARY

Setup 1: (Zero Shot With System):
- Length: 2633 characters
- Tone: [Very fact based and like a reporter]

Setup 2: (Few-Shot With Examples):
- Length: 126 characters
- Tone: [Short and sweet and not very informative]

Setup 3: (User Only):
- Length: 2543 characters
- Tone: [This was a fairly detailed response and much more conversational]

Setup 4: (Creative):
- Length: 2198 characters
- Tone: [This was the best response as every good gold analyst is a conspiracy theorist]



In [30]:
## My observations

### Which setup worked best?
#creative_system

### Key differences I noticed:
#Setup 1: [This took my compute than few shot and I could shape the tone of the response]
#Setup 2: [This was a short and sweet response for the times you want it quick and this was the shortest time]
#Setup 3: [This was a more conversational response and as thorough as the first response]
#Setup 4: [This provided the most interesting response and for newsletter writing would be the most interesting]

### What I learned:
#[There are different ways to manipulate the prompts to get highly different responses.]

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

In [31]:
reasoning_problem = """
The central bank increases the policy rate by 1.5 pp in response to 5 % inflation while nominal wage growth is 3 %.
What happens to real wages?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

To analyze the impact of the central bank's policy rate increase on real wages, we need to consider the relationship between nominal wages, inflation, and real wages.

Real wages can be calculated using the formula:

\[
\text{Real Wage} = \frac{\text{Nominal Wage}}{1 + \text{Inflation Rate}}
\]

In this case:

- Nominal wage growth is 3%, meaning nominal wages increase by 3%.
- Inflation is 5%, and the central bank's policy rate has increased by 1.5 percentage points in response to this inflation. However, for the calculation of real wages, we need the inflation rate, which remains at 5%.

Given this, we can calculate the change in real wages.

1. **Nominal Wage Growth**: If nominal wages grow by 3%, they can be represented as \(1 + 0.03\) or \(1.03\).
2. **Inflation Rate**: The inflation rate is 5%, or \(1 + 0.05\) or \(1.05\).

Now, substituting these values into the real wage formula:

\[
\text{Real Wage} = \frac{1.03}{1.05}
\]

Calculating that gives:

\[
\text{Real Wage} \approx 0.980952
\]

This result indicates that real wages have decreased because the real wage (approximately 0.981) is less than 1. 

### Conclusion:
Real wages decrease due to the inflation rate (5%) outpacing the nominal wage growth (3%). Therefore, despite nominal wages increasing, the purchasing power of those wages falls as inflation erodes their value. Consequently, workers are effectively worse off in terms of real income.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [32]:
list_of_prompts = [
    user_prompt(reasoning_problem + "Think step-by-step about how nominal wages, prices, and interest rates interact through the labor market and aggregate demand. Then explain the real wage effect.")
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

To analyze the effect of an increase in the central bank's policy rate on real wages, we need to understand the relationships among nominal wages, inflation, and interest rates. Let's break this down step-by-step:

### Step 1: Understanding Key Terms
1. **Nominal Wages**: These are the wages that workers receive in current dollars, not adjusted for inflation. In this case, nominal wage growth is 3%.
2. **Inflation**: This represents the rate at which the general price level of goods and services is rising. In this scenario, inflation is at 5%.
3. **Real Wages**: Real wages take into account the purchasing power of nominal wages, adjusting them for inflation. The formula for real wages can be expressed as:
   \[
   \text{Real Wages} = \text{Nominal Wages} - \text{Inflation Rate}
   \]

### Step 2: Analyzing the Situation
1. **Policy Rate Increase**: The central bank has increased the policy rate by 1.5 percentage points in response to inflation. The goal of this action is typically to reduce inflation by making borrowing more expensive, thereby reducing spending in the economy, which could help to curb inflationary pressures over time.
2. **Current Conditions**:
   - Nominal wage growth = 3%
   - Inflation = 5%
   - Policy rate increase = +1.5 pp

### Step 3: Calculating Real Wages
To calculate the effect on real wages, we consider the following:
1. The rate of inflation (5%) exceeds the growth rate of nominal wages (3%). This would mean that the purchasing power of the nominal wages is effectively decreasing.
2. Using the formula for real wages:
   \[
   \text{Real Wages} = \text{Nominal Wage Growth} - \text{Inflation}
   \]
   Substituting in the numbers:
   \[
   \text{Real Wages} = 3\% - 5\% = -2\%
   \]

This means that real wages are decreasing by 2%.

### Step 4: Understanding the Real Wage Effect
1. **Purchasing Power Loss**: Since real wages are negative, it indicates that workers’ purchasing power is declining. They are effectively able to buy less with their income because prices are rising faster than their wages.
2. **Labor Market Response**: As real wages decrease, it could lead to workers requesting higher nominal wages in future negotiations to keep up with rising costs. However, because the central bank has increased interest rates, spending may slow down, which could dampen labor demand and reduce employers' willingness or ability to increase wages.
3. **Aggregate Demand Impact**: Higher interest rates typically lead to lower consumer spending and investment, reducing aggregate demand in the economy. As a result, this could slow down hiring and wage growth further, exacerbating the real wage decline in the future.

### Conclusion
In summary, the central bank's increase in the policy rate does not directly affect nominal wages; however, in the context of rising inflation outpacing nominal wage growth, real wages will decrease by 2%. This loss of purchasing power can lead to dissatisfaction among workers, potentially resulting in demands for higher wages, but it may also slow down economic growth due to reduced aggregate demand from higher interest rates.


## 3. Running Comparative Experiment

We'll test combinations of model type (reasoning vs non-reasoning) and prompting style (zero-shot vs few-shot).


In [33]:
# --------------------------------------------------
# 🧩 Comparing GPT Models: Reasoning vs Non-Reasoning
# --------------------------------------------------

from openai import OpenAI
client = OpenAI()

system_prompt = """
You are an experienced economic analyst.
"""

question = """What is the impact of inflation on real wages? Respond in a concise manner."""

prompt_few = f"""
Use this exact format to answer the question:
Example 1:
{{
  "possible_explanation": "Wage catch-up effect",
  "mechanism": "Workers negotiate higher nominal wages to preserve purchasing power as prices rise.",
  "impact_on_wages": "Nominal wages increase roughly in line with inflation, keeping real wages stable in the short run.",
  "time_frame": "Short to medium run",
  "economic_context": "Inflationary periods with strong labor bargaining power or cost-of-living adjustments."
}}

Example 2:
{{
  "possible_explanation": "Real wage erosion",
  "mechanism": "When nominal wages lag behind price growth, workers lose purchasing power.",
  "impact_on_wages": "Real wages decline despite nominal wage increases, reducing workers’ living standards.",
  "time_frame": "Immediate term",
  "economic_context": "High inflation environments with weak wage indexation or rigid labor contracts."
}}

Now answer:
Q: {question}
"""


# --------------------------------------------------
# MODEL 1: GPT-4-turbo  → Non-Reasoning
# --------------------------------------------------
print("\n==============================")
print("MODEL 1: GPT-4-turbo (Non-Reasoning)")
print("==============================\n")

# Zero-shot
answer_nonreasoning_zero_shot = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ],
)
print("Zero-Shot Prompting (no examples):\n")
print("A:", answer_nonreasoning_zero_shot.choices[0].message.content, "\n")



MODEL 1: GPT-4-turbo (Non-Reasoning)

Zero-Shot Prompting (no examples):

A: Inflation erodes the purchasing power of money. Consequently, when wages do not increase at the same rate as inflation, real wages (which are adjusted for inflation) decline. This leads to a decrease in the actual buying power of individuals, affecting their ability to maintain previous standards of living. Thus, if nominal wage increases do not keep pace with inflation, workers experience a real wage decrease, impacting their economic well-being. 



In [34]:
# --------------------------------------------------
# MODEL 2: GPT-5  → Reasoning
# --------------------------------------------------
print("\n==============================")
print("MODEL 2: GPT-5 (Reasoning-Tuned)")
print("==============================\n")

# Zero-shot
answer_reasoning_zero_shot = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ],
)
print("Zero-Shot Prompting (no examples):\n")
print("A:", answer_reasoning_zero_shot.choices[0].message.content, "\n")



MODEL 2: GPT-5 (Reasoning-Tuned)

Zero-Shot Prompting (no examples):

A: - Real wage = nominal wage divided by the price level. Inflation erodes real wages if prices rise faster than nominal wages.
- If nominal wages at least match inflation (and ideally reflect productivity growth), real wages are maintained or rise; if they lag, purchasing power falls.
- Unexpected inflation often cuts real wages temporarily under fixed contracts; cost-of-living adjustments and indexation mitigate this.
- Adjustment is uneven: lower-wage, less-unionized workers usually see larger real wage declines; strong labor markets and bargaining power help wages catch up. 



In [38]:
print("\n==============================")
print("MODEL 1: GPT-4-turbo (Non-Reasoning)")
print("==============================\n")

# Few-shot
answer_nonreasoning_few_shot = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_few}
    ],
)
print("Few-Shot Prompting (with examples):\n")
print("A:", answer_nonreasoning_few_shot.choices[0].message.content, "\n")


MODEL 1: GPT-4-turbo (Non-Reasoning)

Few-Shot Prompting (with examples):

A: {
  "possible_explanation": "Inflationary impact on real wages",
  "mechanism": "As consumer prices increase, nominal wages may not adjust at the same rate, leading to decreased purchasing power of the wages.",
  "impact_on_wages": "Real wages decrease if nominal wage increases do not keep pace with the inflation rate.",
  "time_frame": "Short to medium term",
  "economic_context": "Periods of rapid inflation where nominal wage adjustments lag behind the rate of inflation."
} 



In [39]:
print("\n==============================")
print("MODEL 2: GPT-5 (Reasoning-Tuned)")
print("==============================\n")

# Few-shot
answer_reasoning_few_shot = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_few}
    ],
)
print("Few-Shot Prompting (with examples):\n")
print("A:", answer_reasoning_few_shot.choices[0].message.content, "\n")


MODEL 2: GPT-5 (Reasoning-Tuned)

Few-Shot Prompting (with examples):

A: Example 1:
{
  "possible_explanation": "Wage catch-up/indexation",
  "mechanism": "Cost-of-living adjustments or strong bargaining raise nominal pay alongside prices.",
  "impact_on_wages": "Real wages stay roughly flat; nominal wages rise with inflation.",
  "time_frame": "Short to medium run",
  "economic_context": "Inflation with indexed contracts, tight labor markets, or robust unions."
}

Example 2:
{
  "possible_explanation": "Real wage squeeze",
  "mechanism": "Prices rise faster than nominal pay due to weak indexation or delayed bargaining.",
  "impact_on_wages": "Real wages fall, eroding purchasing power.",
  "time_frame": "Immediate to near term",
  "economic_context": "High or accelerating inflation with slack labor markets or rigid pay-setting."
} 




## 4. Evaluation Framework

LLM as a judge


In [40]:
import json

# --------------------------------------------------
# ⚖️ LLM-as-a-Judge Evaluation Script
# --------------------------------------------------

# Define evaluation scale (0–4)
# 0 = completely incorrect / irrelevant
# 1 = partially correct but weak or inaccurate reasoning
# 2 = fair factual accuracy, minimal reasoning
# 3 = accurate and somewhat reasoned
# 4 = highly accurate, clear causal explanation, correct logic

evaluation_prompt = f"""
You are an impartial economics teacher grading two student answers to the same question.

Question:
{question}

Answer A (non-reasoning model):
{answer_nonreasoning_few_shot.choices[0].message.content}

Answer B (reasoning model):
{answer_reasoning_few_shot.choices[0].message.content}

Evaluate both answers on accuracy and reasoning quality on a 0–4 scale:
- 0 = completely incorrect or irrelevant
- 1 = partially correct, but flawed
- 2 = fair factual accuracy, limited reasoning
- 3 = mostly correct, some reasoning
- 4 = fully accurate and clearly reasoned, ability to see the interdependencies between variables.

Return your evaluation as a JSON object in this exact format:
{{
  "Answer A Score": <0-4>,
  "Answer B Score": <0-4>,
  "Better Answer": "A" or "B",
  "Explanation": "Why the better answer is more accurate or reasoned"
}}
"""

# Choose a strong evaluator model (GPT-4.1 is good for judging)
evaluation = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "system", "content": "You are an impartial LLM evaluator for economics-related answers."},
        {"role": "user", "content": evaluation_prompt}
    ],
)

# Parse and display the evaluation
response_text = evaluation.choices[0].message.content

# Optional: try to parse JSON for structured output
try:
    result = json.loads(response_text)
    print("\nParsed JSON Result:")
    print(json.dumps(result, indent=2))
except json.JSONDecodeError:
    print("\nNote: Could not parse JSON, model may have returned free text instead.")



Parsed JSON Result:
{
  "Answer A Score": 3,
  "Answer B Score": 4,
  "Better Answer": "B",
  "Explanation": "Answer B is more accurate and better reasoned because it presents the key alternative outcomes and mechanisms: when nominal wages are indexed or bargaining is strong real wages can keep pace with inflation, while when nominal pay lags (due to weak indexation or slack labor markets) real wages fall. It links these outcomes to time frames and economic context, showing interdependencies between inflation, wage-setting institutions, and labor market strength. Answer A is correct in describing the common case where real wages decline if nominal wages do not keep up, but it is one-sided and lacks the alternative scenarios and mechanisms that Answer B includes."
}


### 🏗️ Activity #2:

Evaluate different prompting strategies using your own example.

In [41]:
# 🏗️ Activity #2: Evaluate Different Prompting Strategies
# Create your own economics question and compare approaches

import json
from openai import OpenAI
client = OpenAI()

# --------------------------------------------------
# Define Your Custom Question
# --------------------------------------------------
custom_question = """
Explain the relationship between gold and monetary debasement
"""

print("="*60)
print("ACTIVITY #2: Custom Evaluation Experiment")
print("="*60)
print(f"\nQuestion: {custom_question}\n")

# --------------------------------------------------
# Approach 1: Simple Zero-Shot
# --------------------------------------------------
print("\n🔵 Approach 1: Simple Zero-Shot")
print("-" * 60)

approach_1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": custom_question}
    ]
)

print(approach_1.choices[0].message.content)

# --------------------------------------------------
# Approach 2: Chain-of-Thought Prompting
# --------------------------------------------------
print("\n\n🟢 Approach 2: Chain-of-Thought Prompting")
print("-" * 60)

cot_prompt = f"""
{custom_question}

Let's think through this step-by-step:
1. First, describe why gold is considered a safe haven
2. Then, explain the relationship between monetary debasement and the price performance of gold
3. Next, discuss historical evidence from the 20th century
4. Finally, analyze how the relationship has changed in the last two decades
"""

approach_2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": cot_prompt}
    ]
)

print(approach_2.choices[0].message.content)

# --------------------------------------------------
# Approach 3: Expert Persona with Examples
# --------------------------------------------------
print("\n\n🟡 Approach 3: Expert Persona with Few-Shot Examples")
print("-" * 60)

expert_system = "You are a macroeconomics professor known for clear, evidence-based explanations."

few_shot_phillips = f"""
Example 1:
Q: Why do people like gold?
A: People like gold because it's nobody's liability

Example 2:
Q: Why do governments practice monetary debasement?
A: When the spending needs of the government grow extreme and its politically unpopular to cut fiscal spending governments devalue their debts.

Now answer:
Q: {custom_question}
"""

approach_3 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": expert_system},
        {"role": "user", "content": few_shot_phillips}
    ]
)

print(approach_3.choices[0].message.content)

# --------------------------------------------------
# Approach 4: Reasoning Model
# --------------------------------------------------
print("\n\n🔴 Approach 4: Reasoning Model (GPT-5)")
print("-" * 60)

approach_4 = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "system", "content": "You are a monetarist economist. Provide detailed causal reasoning."},
        {"role": "user", "content": custom_question}
    ]
)

print(approach_4.choices[0].message.content)

# --------------------------------------------------
# LLM-as-a-Judge Evaluation
# --------------------------------------------------
print("\n\n" + "="*60)
print("⚖️ LLM-AS-A-JUDGE EVALUATION")
print("="*60)

evaluation_prompt = f"""
You are evaluating four different AI responses to the same economics question.

Question:
{custom_question}

Response 1 (Simple Zero-Shot):
{approach_1.choices[0].message.content}

Response 2 (Chain-of-Thought):
{approach_2.choices[0].message.content}

Response 3 (Expert + Few-Shot):
{approach_3.choices[0].message.content}

Response 4 (Reasoning Model):
{approach_4.choices[0].message.content}

Evaluate each response on:
1. **Accuracy** (0-4): Factual correctness and economic validity
2. **Reasoning Quality** (0-4): Logical flow and causal explanations
3. **Completeness** (0-4): Coverage of key concepts (traditional vs modern Phillips Curve)
4. **Clarity** (0-4): Accessibility and structure

Return a JSON object with this structure:
{{
  "Response_1": {{
    "accuracy": <score>,
    "reasoning": <score>,
    "completeness": <score>,
    "clarity": <score>,
    "total": <sum>,
    "strengths": "...",
    "weaknesses": "..."
  }},
  "Response_2": {{ ... }},
  "Response_3": {{ ... }},
  "Response_4": {{ ... }},
  "best_overall": "Response X",
  "ranking": ["Response X", "Response Y", ...],
  "conclusion": "Summary of why the best response excels"
}}
"""

judge = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "system", "content": "You are an impartial AI judge evaluating economic explanations."},
        {"role": "user", "content": evaluation_prompt}
    ]
)

judge_response = judge.choices[0].message.content

# Try to parse and pretty-print the JSON
try:
    # Remove markdown code blocks if present
    clean_response = judge_response.replace("```json", "").replace("```", "").strip()
    evaluation_result = json.loads(clean_response)
    print("\n📊 Evaluation Results:\n")
    print(json.dumps(evaluation_result, indent=2))
    
    # Print summary
    print("\n" + "="*60)
    print("🏆 FINAL VERDICT")
    print("="*60)
    print(f"\nBest Response: {evaluation_result.get('best_overall', 'N/A')}")
    print(f"\nRanking: {' > '.join(evaluation_result.get('ranking', []))}")
    print(f"\nConclusion: {evaluation_result.get('conclusion', 'N/A')}")
    
except json.JSONDecodeError:
    print("\nRaw Evaluation (could not parse JSON):")
    print(judge_response)

ACTIVITY #2: Custom Evaluation Experiment

Question: 
Explain the relationship between gold and monetary debasement



🔵 Approach 1: Simple Zero-Shot
------------------------------------------------------------
The relationship between gold and monetary debasement is rooted in the historical role of gold as a form of currency and a store of value. Here's a detailed explanation:

### Historical Context
1. **Gold as Currency**: Historically, many civilizations used gold as a form of money due to its intrinsic properties—it's durable, divisible, and has a universally accepted value. This made it a convenient medium for trade.

2. **Debasement Defined**: Monetary debasement refers to the reduction in the value of currency, often achieved by decreasing the metal content of coins or through excessive money printing. This practice effectively erodes the purchasing power of money.

### Relationship Between Gold and Monetary Debasement

1. **Value Retention**: Gold has been seen as a safe haven

## Saving results

In [42]:
# Create markdown content
markdown_content = f"""
# 🧠 Reasoning Model Answer
### Question:
How does inflation affect interest rates and the broader market?

### Model Used:
`gpt-4.1` (Reasoning-tuned)

### Response:
{answer_reasoning_few_shot.choices[0].message.content}

---

*This answer was generated by a reasoning model to illustrate step-by-step economic reasoning.*
"""

output_path='./results.md'
# Save to file
with open(output_path, "w", encoding="utf-8") as f:
    f.write(markdown_content)

print(f"✅ Reasoning model answer saved to: {os.path.abspath(output_path)}")

✅ Reasoning model answer saved to: /home/omar/makerspace/MakerSpace/Session_01_LLM_APIs_&_AI-Assisted_Development/results.md


## Conclusion

- **Few-shot prompts** improve structure and reasoning consistency.  
- **Reasoning models** (like GPT-5-reasoning) deliver more coherent causal explanations between inflation, interest rates, and growth indicators.  
- **Non-reasoning models** (e.g., GPT-5-mini) provide faster, surface-level insights ideal for retrieval or summarization tasks.  
- Future work could add **RAG pipelines** with real-time macroeconomic data or integrate with financial dashboards for live LLM reasoning visualization.